## Welcome to the NEXRAD notebook!

#### **Intent:** This notebook will find and visualize NEXRAD data for a requested radar, date, time, and product. 
#### **Audience:** Anyone with at least 4GB of memory on their computer or computing environment. No programming experience required to go through this notebook, but it will help you understand where this data is coming from.     
#### **Outcome:** A plot and animation of NEXRAD imagery for a selected radar, date, time range, and product.          

The Next Generation Weather Radar (NEXRAD) system is a network of 160 high-resolution S-band Doppler weather radars jointly operated by the National Weather Service (NWS), the Federal Aviation Administration (FAA), and the U.S. Air Force. The NEXRAD system detects precipitation and wind, and its data can be processed to map precipitation patterns and movement. NCEI provides access to archived NEXRAD Level-II data and Level-III products.

This notebook will use NEXRAD data to visualize radar imagery from one specific, user-selected radar, and display the results from the date/time range in an animation. 

## Importing

In [ ]:
import ipywidgets as widgets
import requests
import netCDF4
import s3fs
import numpy as np
from matplotlib import animation
import cartopy.crs as ccrs
from cartopy import feature as cf
import xarray
from datetime import datetime
import matplotlib.pyplot as plt
import tempfile
import pytz
from datetime import datetime
import pyart
import pandas as pd
import nexradaws

conn = nexradaws.NexradAwsInterface()
templocation = tempfile.mkdtemp()

## Definitions

In [ ]:
def aws_date():
    
    global year, month, day, shour, smin, ehour, emin
    
    aws = s3fs.S3FileSystem(anon=True)
    years = aws.ls(f'noaa-nexrad-level2/', refresh=True)  

    year_options = []

    for y in years:
        year_options.append(y[-4:])

    # Formatting settings for drop-down menus
    style = {'description_width':'120px'}
    layout = widgets.Layout(width='325px')
    
    year = widgets.Dropdown(options=[y for y in year_options], description='Year:', style=style, layout=layout)
    month = widgets.Dropdown(options=[('Jan', '01'), ('Feb', '02'), ('Mar', '03'), ('Apr', '04'), ('May', '05'), ('Jun', '06'), ('Jul', '07'), ('Aug', '08'), ('Sep', '09'), ('Oct', '10'), ('Nov', '11'), ('Dec', '12')], description='Month:', style=style, layout=layout)
    day = widgets.Dropdown(options=[('01'), ('02'), ('03'), ('04'), ('05'), ('06'), ('07'), ('08'), ('09'), ('10'), ('11'), ('12'), ('13'), ('14'), ('15'), ('16'), ('17'), ('18'), ('19'), ('20'), ('21'), ('22'), ('23'), ('24'), ('25'), ('26'), ('27'), ('28'), ('29'), ('30'), ('31')], description='Day:', style=style, layout=layout)
    shour = widgets.Dropdown(options=[('00'), ('01'), ('02'), ('03'), ('04'), ('05'), ('06'), ('07'), ('08'), ('09'), ('10'), ('11'), ('12'), ('13'), ('14'), ('15'), ('16'), ('17'), ('18'), ('19'), ('20'), ('21'), ('22'), ('23')], description='Start Hour:', style=style, layout=layout)
    smin = widgets.Dropdown(options=[('00'), ('01'), ('02'), ('03'), ('04'), ('05'), ('06'), ('07'), ('08'), ('09'), ('10'), ('11'), ('12'), ('13'), ('14'), ('15'), ('16'), ('17'), ('18'), ('19'), ('20'), ('21'), ('22'), ('23'), ('24'), ('25'), ('26'), ('27'), ('28'), ('29'), ('30'), ('31'), ('32'), ('33'), ('34'), ('35'), ('36'), ('37'), ('38'), ('39'), ('40'), ('41'), ('42'), ('43'), ('44'), ('45'), ('46'), ('47'), ('48'), ('49'), ('50'), ('51'), ('52'), ('53'), ('54'), ('55'), ('56'), ('57'), ('58'), ('59')], description='Start Minutes:', style=style, layout=layout)
    ehour = widgets.Dropdown(options=[('00'), ('01'), ('02'), ('03'), ('04'), ('05'), ('06'), ('07'), ('08'), ('09'), ('10'), ('11'), ('12'), ('13'), ('14'), ('15'), ('16'), ('17'), ('18'), ('19'), ('20'), ('21'), ('22'), ('23')], description='End Hour:', style=style, layout=layout)
    emin = widgets.Dropdown(options=[('00'), ('01'), ('02'), ('03'), ('04'), ('05'), ('06'), ('07'), ('08'), ('09'), ('10'), ('11'), ('12'), ('13'), ('14'), ('15'), ('16'), ('17'), ('18'), ('19'), ('20'), ('21'), ('22'), ('23'), ('24'), ('25'), ('26'), ('27'), ('28'), ('29'), ('30'), ('31'), ('32'), ('33'), ('34'), ('35'), ('36'), ('37'), ('38'), ('39'), ('40'), ('41'), ('42'), ('43'), ('44'), ('45'), ('46'), ('47'), ('48'), ('49'), ('50'), ('51'), ('52'), ('53'), ('54'), ('55'), ('56'), ('57'), ('58'), ('59')], description='End Minutes:', style=style, layout=layout)

    start_time = widgets.HBox([shour, smin])
    end_time = widgets.HBox([ehour, emin])

    print("Select your chosen date, then run the next cell.")
    display(year, month, day)
    display(start_time, end_time)

In [ ]:
def aws_radar(year, month, day):
    
    global radar, product
    
    aws = s3fs.S3FileSystem(anon=True)
    radars = aws.ls(f'noaa-nexrad-level2/{year}/{month}/{day}/', refresh=True)
    
    radar_options = []
    for r in radars:
        radar_options.append(r[-4:])
        
    # Formatting settings for drop-down menus
    style = {'description_width':'120px'}
    layout = widgets.Layout(width='325px')        
    radar = widgets.Dropdown(options=[r for r in radar_options], description='Radar:', style=style, layout=layout)
    product = widgets.Dropdown(options=[("Reflectivity", "reflectivity"), ("Velocity", "velocity")], description='Radar Product:', style=style, layout=layout)
    display(radar, product)

In [ ]:
def aws_data(year, month, day, shour, smin, ehour, emin, radar):
    
    global results 
    global scans
    
    start = pd.Timestamp(year,month,day,shour,smin).tz_localize(tz='UTC')
    end = pd.Timestamp(year,month,day,ehour,emin).tz_localize(tz='UTC')

    scans = conn.get_avail_scans_in_range(start, end, radar)
    print("There are {} scans available between {} and {}\n".format(len(scans), start, end))
    
    print("Downloading the first 4 scans...")
    results = conn.download(scans[0:4], templocation)

In [ ]:
def aws_plot(product):
    if product == "reflelctivity":
        product_num = 0
    elif product == "velocity":
        product_num = 1
    
    fig = plt.figure(figsize=(16,12))
    
    for i,scan in enumerate(results.iter_success(),start=1):
        ax = fig.add_subplot(2,2,i)
        openradar = scan.open_pyart()
        display = pyart.graph.RadarDisplay(openradar)
        display.plot(product,product_num,ax=ax,title="{} {}".format(scan.radar_id,scan.scan_time))
        display.set_limits((-150, 150), (-150, 150), ax=ax)

In [ ]:
def aws_animate():
    if product == "reflelctivity":
        product_num = 0
    elif product == "velocity":
        product_num = 1
        
    fig, ax = plt.subplots()
    def animate(i):
        for scan_count in enumerate(results.iter_success()):
            if scan_count[0] == i:
                scan = scan_count[1]
                fig.clear()
                ax = fig.add_subplot()
                radar = scan.open_pyart()
                display = pyart.graph.RadarDisplay(radar)
                display.plot(product,product_num,ax=ax,title="{} {}".format(scan.radar_id,scan.scan_time))
                display.set_limits((-150, 150), (-150, 150), ax=ax)

    ani = animation.FuncAnimation(fig, animate, repeat=True, frames=len(scans), interval=200)
    ani.save('radar_animation.gif', writer='pillow')
    plt.close("all")
    plt.close(fig)

## Variable Selection & Visualization

In [ ]:
aws_date()

In [ ]:
aws_radar(year.value, month.value, day.value)

In [ ]:
aws_data(int(year.value), int(month.value), int(day.value), int(shour.value), int(smin.value), int(ehour.value), int(emin.value), radar.value)

In [ ]:
aws_plot(product.value)

In [ ]:
aws_animate()

**Product Documentation:** 
- https://www.ncei.noaa.gov/products/radar/next-generation-weather-radar (NCEI product page)
- https://www.weather.gov/nl2/NEXRADView (Radar view)
- https://www.roc.noaa.gov/WSR88D/ (Radar operations center)

**Coding References:**    
- https://nexradaws.readthedocs.io/en/latest/Tutorial.html     
- https://github.com/russ-schumacher/ats641_spring2022/blob/master/example_notebooks/pyart_nexrad_maps_reports.ipynb
- https://arm-doe.github.io/pyart/examples/io/plot_nexrad_data_aws.html

**CSP Access:**     

- NEXRAD Level 2 Real-Time and Archive Data     
    - AWS: https://registry.opendata.aws/noaa-nexrad/
    - Google: https://console.cloud.google.com/marketplace/product/noaa-public/nexrad-l2

- NEXRAD Level 3 Real-Time and Archive Data    
    - Google: https://console.cloud.google.com/marketplace/details/noaa-public/nexrad-l3

The unique component of this Jupyter notebook is that you are not requried to download any datasets -- all data will be pulled directly from the AWS cloud. You can learn more about NOAA's efforts to move more data to the cloud at this site: https://www.noaa.gov/information-technology/open-data-dissemination. As we continute to make more data widely accessible on the cloud, we'll also create more Jupyter notebooks like this one, so anyone can visualize weather and climate data without any cost or restriction. 